In [82]:
import pickle
import sys
sys.path.append('..')
with open('../data/tt_subset.pkl','rb') as file:
    data = pickle.load(file)

RDKit WARNING: [02:34:57] Enabling RDKit 2019.09.3 jupyter extensions


In [83]:
X, y = data['graph'], data['tt']

In [84]:
from app.kernel import *
from app.smiles import *
from app.ActiveLearning import *
from app.Nystrom import NystromGaussianProcessRegressor
kernel_config = KernelConfig(save_mem=False, property='tt')

In [205]:
from sklearn.gaussian_process import GaussianProcessRegressor
model = GaussianProcessRegressor(kernel=kernel_config.kernel, random_state=0, optimizer=None,alpha=0.000001).fit(X[1:], y[1:])

In [206]:
def predict_manual_loocv(i):
    train_X = np.delete(np.array(X), i)
    train_y = np.delete(np.array(y), i)
    model = GaussianProcessRegressor(kernel=kernel_config.kernel, random_state=0, optimizer=None,
                                             alpha=0.000001).fit(train_X, train_y)
    y_pred, y_std = model.predict(X, return_std=True)
    return y_pred[i], y_std[i]

In [207]:
model.predict((X[0],X[0]), return_std=True)

(array([221.42360253, 221.42355472]), array([0.02284074, 0.02284309]))

In [215]:
import scipy
K = model.kernel_(X)
#y_ = y - y.mean()
y_ = y
K[np.diag_indices_from(K)] += 0.000001
I_mat = np.eye(K.shape[0])

L_ = scipy.linalg.cholesky(K, lower=True)
L_inv = scipy.linalg.solve_triangular(L_.T, np.eye(L_.shape[0]))
K_inv = L_inv.dot(L_inv.T)

#K_inv = scipy.linalg.cho_solve(scipy.linalg.cho_factor(K,lower=True), I_mat)
#K_inv = np.linalg.inv(K) 
y_pred = y_ - K_inv.dot(y_) / K_inv.diagonal() #+ y.mean()
y_std = np.sqrt(1/ K_inv.diagonal())

In [216]:
# using loocv formula
y_pred, y_std

(0    221.423499
 1    192.282991
 2    231.914260
 3    148.011123
 4    237.226565
 Name: tt, dtype: float64,
 array([0.0228698 , 0.02862989, 0.01309133, 0.5390026 , 0.53863297]))

In [217]:
# using manual loop
manual_y = np.array([predict_manual_loocv(i) for i in range(5)])

In [218]:
df = pd.DataFrame({'Smiles':list(map(lambda x:x.smiles, X)), 'y':y, 'y_pred_loocv':y_pred, 'y_pred_loop':manual_y[:,0],'y_std_loocv':y_std, 'y_std_loop':manual_y[:,1]})

In [219]:
df

,Smiles,y,y_pred_loocv,y_pred_loop,y_std_loocv,y_std_loop
0,CCCCCCCCCC,243.536,221.423499,221.423547,0.022870,0.022846
1,CCCCCCCC,216.418,192.282991,192.281035,0.028630,0.028607
2,CCCCCCCCC,219.680,231.914260,231.914260,0.013091,0.013053
3,c1ccccc1,278.674,148.011123,148.011011,0.539003,0.539002
4,Cc1ccccc1,178.188,237.226565,237.225673,0.538633,0.538632


In [161]:
x_1d = np.array([[1],[1.5], [3], [4], [10]])
y_1d = np.array([[1],[3],[10],[17],[100]])

In [12]:
x_1d.shape

(5, 1)

In [162]:
from sklearn.gaussian_process import GaussianProcessRegressor
model = GaussianProcessRegressor( random_state=0, normalize_y=True, alpha=0.01).fit(x_1d[:4], y_1d[:4])

In [163]:
import numpy as np
def predict_manual_loocv_(i):
    train_X = np.delete(np.array(x_1d), i).reshape(-1,1)
    train_y = np.delete(np.array(y_1d), i).reshape(-1,1)
    model = GaussianProcessRegressor(random_state=0, optimizer=None, alpha=0.01).fit(train_X, train_y)
    y_pred, y_std = model.predict(x_1d, return_std=True)
    return y_pred[i], y_std[i]

In [164]:
manual_y = np.array([predict_manual_loocv_(i) for i in range(5)])

In [165]:
model = GaussianProcessRegressor(random_state=0, optimizer=None, normalize_y=True, alpha=0.01).fit(x_1d, y_1d)

In [ ]:
L_inv = solve_triangular( self.L_.T, np.eye(self.L_.shape[0]) )

In [172]:
import scipy
K = model.kernel_(x_1d)
#y_1d_ = y_1d - y_1d.mean()
K[np.diag_indices_from(K)] += 0.01
I_mat = np.eye(K.shape[0])

L_ = scipy.linalg.cholesky(K, lower=True)
L_inv = scipy.linalg.solve_triangular(L_.T, np.eye(L_.shape[0]))
K_inv = L_inv.dot(L_inv.T)

#K_inv = scipy.linalg.cho_solve(scipy.linalg.cho_factor(K,lower=True), I_mat)
#K_inv = np.linalg.inv(K) 
y_pred = y_1d - K_inv.dot(y_1d) / K_inv.diagonal().reshape(-1,1) 
y_std = np.sqrt(1/ K_inv.diagonal())

In [175]:
model.L_

array([[1.00498756e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [8.78117238e-01, 4.88784324e-01, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [1.34663640e-01, 4.22276234e-01, 9.01969227e-01, 0.00000000e+00,
        0.00000000e+00],
       [1.10538647e-02, 7.00315922e-02, 6.38014482e-01, 7.73246991e-01,
        0.00000000e+00],
       [2.56396915e-18, 4.14182075e-16, 2.53857588e-11, 1.96751923e-08,
        1.00498756e+00]])

In [174]:
L_

array([[1.00498756e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [8.78117238e-01, 4.88784324e-01, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [1.34663640e-01, 4.22276234e-01, 9.01969227e-01, 0.00000000e+00,
        0.00000000e+00],
       [1.10538647e-02, 7.00315922e-02, 6.38014482e-01, 7.73246991e-01,
        0.00000000e+00],
       [2.56396915e-18, 4.14182075e-16, 2.53857588e-11, 1.96751923e-08,
        1.00498756e+00]])

In [167]:
manual_y

array([[array([2.37304219]), 0.4435945582622642],
       [array([1.35730491]), 0.41574440015315856],
       [array([11.52474496]), 0.6884927626672924],
       [array([5.99506877]), 0.7667534864542424],
       [array([2.80245391e-07]), 0.9999999999999998]], dtype=object)

In [173]:
y_std

array([0.45472644, 0.42760192, 0.6957171 , 0.77324699, 1.00498756])

In [168]:
y_std

array([0.45472644, 0.42760192, 0.6957171 , 0.77324699, 1.00498756])

In [55]:
y_1d - K_inv.dot(y_1d) / K_inv.diagonal().reshape(-1,1)

array([[2.37304219e+00],
       [1.35730491e+00],
       [1.15247450e+01],
       [5.99506877e+00],
       [2.80245388e-07]])

In [73]:
y_1d.mean()

26.2